# Lab 12: Path Planning

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

### <span style="color:rgb(0,150,0)">The notebook only provides skeleton code for you to integrate the Localization class with the Real Robot.</span>

<hr>

In [150]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
import pathlib
import os
from utils import load_config_params
from localization_extras import Localization
import time

# The imports below will only work if you copied the required ble-related python files 
# into the notebooks directory
from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import numpy as np

# Setup Logger
LOG = get_logger('demo_notebook.log')
LOG.propagate = False

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2024-05-14 16:36:06,645 | INFO     |: Logger demo_notebook.log already initialized.
2024-05-14 16:36:06,646 | INFO     |: GUI is already running. Shutdown notebook to force restart the GUI.


In [151]:
# Start the plotter
START_PLOTTER()

2024-05-14 16:36:06,847 | ERROR    |: Plotter is already running


# The RealRobot class
Define the RealRobot class in the code cell below, based on the documentation and your real robot communication protocol. <br>
This class is used by the **Localization** class to communicate with the real robot. <br>
More specifically, the **Localization** class utilizes the **RealRobot's** member function **perform_observation_loop()** to get the 18 sensor readings and store them in its member variable **obs_range_data**, which is then utilized in the update step.

In [152]:
class RealRobot():
    """A class to interact with the real robot
    """
    def __init__(self, commander, ble):
        # Load world config
        self.world_config = os.path.join(str(pathlib.Path(os.getcwd()).parent), "config", "world.yaml")
        
        self.config_params = load_config_params(self.world_config)
        
        # Commander to commuincate with the Plotter process
        # Used by the Localization module to plot odom and belief
        self.cmdr = commander

        # ArtemisBLEController to communicate with the Robot
        self.ble = ble

    def get_pose(self):
        """Get robot pose based on odometry
        
        Returns:
            current_odom -- Odometry Pose (meters, meters, degrees)
        """
        # raise NotImplementedError("get_pose is not implemented")
        
        
    def mapping_final_angle_notification_handler(self, uuid, char_bytearr):
        global robot_angles, tof1_mapping_readings, tof2_mapping_readings
        string = self.ble.bytearray_to_string(char_bytearr)
        parts = string.split("|")  
        print(parts)
        robot_angles.append(float(parts[0][4:]))
        tof1_mapping_readings.append(float(parts[1][5:]))
        tof2_mapping_readings.append(float(parts[2][5:]))



    def perform_observation_loop(self, rot_vel=120):
        """Perform the observation loop behavior on the real robot, where the robot does  
        a 360 degree turn in place while collecting equidistant (in the angular space) sensor
        readings, with the first sensor reading taken at the robot's current heading. 
        The number of sensor readings depends on "observations_count"(=18) defined in world.yaml.
        
        Keyword arguments:
            rot_vel -- (Optional) Angular Velocity for loop (degrees/second)
                        Do not remove this parameter from the function definition, even if you don't use it.
        Returns:
            sensor_ranges   -- A column numpy array of the range values (meters)
            sensor_bearings -- A column numpy array of the bearings at which the sensor readings were taken (degrees)
                               The bearing values are not used in the Localization module, so you may return a empty numpy array
        """
        # initialize the PID parameters
        self.ble.send_command(CMD.PID_STATUS_INI, "1")
        self.ble.send_command(CMD.SET_ORIENT_PID_VALUES, "|3|0.01|0.2")
        
        # rotation
        print("Start Observation Loop")
        self.ble.send_command(CMD.PID_ANGLE_INCREMENT, "")
        
        # send data (only use the front TOF data)
        global robot_angles, tof1_mapping_readings, tof2_mapping_readings
        robot_angles = []
        tof1_mapping_readings = []
        tof2_mapping_readings = []
        self.ble.send_command(CMD.SEND_PID_ANGLE_INCREMENT_DEBUG_READINGS, "1")
        print("Start Sending Data!")
        
        while len(robot_angles) < 18 or (len(tof1_mapping_readings) < 18*10):   # if the data has not been all collected, just wait
            asyncio.run(asyncio.sleep(2))
        
        print("Finish Data Transmission")
        observations_count=self.config_params["mapper"]["observations_count"] # 18

        sensor_ranges = []
        sensor_bearings = []
        
        # get the unique elements and keep the raw order of the elements
        robot_angles_unique_elements, robot_angles_unique_indices = np.unique(robot_angles, return_index=True)
        robot_angles_sorted_indices = np.argsort(robot_angles_unique_indices)
        robot_angles = robot_angles_unique_elements[robot_angles_sorted_indices]
        
        # take the stable tof readings and keep the avg value as the final readings
        def remove_outliers(readings):
            q1 = np.percentile(readings, 25)
            q3 = np.percentile(readings, 75)
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr
            
            # print(readings)
            filtered_readings = [x for x in readings if lower_bound <= x and x <= upper_bound]
            return filtered_readings

        def calculate_average(readings):
            stable_readings = remove_outliers(readings)
            if stable_readings:
                return np.mean(stable_readings)
            else:
                return None  # Or handle the case where all readings are considered outliers


        tof1_mapping_readings = np.array(tof1_mapping_readings).reshape(18, 10)
        tof2_mapping_readings = np.array(tof2_mapping_readings).reshape(18, 10)
        
        for i in range(np.shape(tof1_mapping_readings)[0]):
            tof1_mapping_readings[i] = calculate_average(tof1_mapping_readings[i])
            tof2_mapping_readings[i] = calculate_average(tof2_mapping_readings[i])

        if (len(np.shape(tof1_mapping_readings)) > 1):
            tof1_mapping_readings = np.mean(tof1_mapping_readings, axis=1)
            tof2_mapping_readings = np.mean(tof2_mapping_readings, axis=1)
            
        tof1_mapping_readings = tof1_mapping_readings.tolist()
        tof2_mapping_readings = tof2_mapping_readings.tolist()
        robot_angles = robot_angles.tolist()
        
        # re-order the data list: [20, 360] -> [0, 340]
        robot_angles.insert(0, robot_angles[-1])
        robot_angles = robot_angles[:-1]
        tof1_mapping_readings.insert(0, tof1_mapping_readings[-1])
        tof1_mapping_readings = tof1_mapping_readings[:-1]
        tof2_mapping_readings.insert(0, tof2_mapping_readings[-1])
        tof2_mapping_readings = tof2_mapping_readings[:-1]
        robot_angles[0] = 0 # 360 -> 0
        
        for i in range(observations_count):
            sensor_ranges.append(tof1_mapping_readings[i]/1000)
            sensor_bearings.append(robot_angles[i])

        sensor_ranges = np.array(sensor_ranges).reshape(-1, 1)
        sensor_bearings = np.array(sensor_bearings).reshape(-1, 1)
        
        print("Finish observation")
        
        return sensor_ranges,sensor_bearings

In [153]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2024-05-14 16:36:10,105 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:42:30:78:92:49
2024-05-14 16:36:12,702 | INFO     |: Connected to c0:42:30:78:92:49


In [154]:
# for collecting the data
robot_angles = []
tof1_mapping_readings = []
tof2_mapping_readings = []

In [155]:
# Initialize RealRobot with a Commander object to communicate with the plotter process
# and the ArtemisBLEController object to communicate with the real robot
robot = RealRobot(cmdr, ble)

ble.start_notify(ble.uuid["RX_STRING"], robot.mapping_final_angle_notification_handler)


# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a RealRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2024-05-14 16:36:14,321 | INFO     |:  | Number of observations per grid cell: 18
2024-05-14 16:36:14,322 | INFO     |:  | Precaching Views...
2024-05-14 16:36:15,536 | INFO     |:  | Precaching Time: 1.214 secs
2024-05-14 16:36:15,536 | INFO     |: Initializing beliefs with a Uniform Distribution
2024-05-14 16:36:15,537 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


In [156]:
def get_cur_pose():
    print("Get cur pose ing")
    
    loc.get_observation_data()  # rotate 360 degrees to get the observation
    
    loc.update_step()   # Update the probabilities in self.bel based on self.bel_bar and the sensor model.
    
    loc.plot_update_step_data(plot_data=True)   # plot the update 
    
    bel_argmax = get_max(loc.bel)
    cur_bel = loc.mapper.from_map(*bel_argmax[0])   # Return the continuous world coordinates (x,y,z) [in (m,m,deg)] of the center of the grid cell (cx, cy, cz)
    return cur_bel
    
def compute_control(cur_pose, prev_pose):
    """ Given the current and previous odometry poses, this function extracts
    the control information based on the odometry motion model.

    Args:
        cur_pose  ([Pose]): Current Pose
        prev_pose ([Pose]): Previous Pose 

    Returns:
        [delta_rot_1]: Rotation 1  (degrees)
        [delta_trans]: Translation (meters)
        [delta_rot_2]: Rotation 2  (degrees)
    """
    
    print("Compute Ctrl ing")
    
    del_x = np.transpose(cur_pose)[0] - np.transpose(prev_pose)[0]
    del_y = np.transpose(cur_pose)[1] - np.transpose(prev_pose)[1]

    normalizer = np.vectorize(lambda x : mapper.normalize_angle(x))  # use np.vectorize to let the lambda function apply elementwise operation
    # print(np.arctan2(del_y,del_x))
    # print(np.transpose(prev_pose))
    # print(np.transpose(prev_pose)[2])
    
    delta_rot_1 = np.degrees(np.arctan2(del_y,del_x) - np.transpose(prev_pose)[2]) #initial rotation
    delta_trans = np.sqrt(np.power(del_x,2) + np.power(del_y,2)) #translation
    delta_rot_2 = np.transpose(cur_pose)[2] - np.transpose(prev_pose)[2] - delta_rot_1 #final rotation
    
    print("Finish Compute Control")
    
    return np.array([ normalizer(delta_rot_1), delta_trans, normalizer(delta_rot_2)]).T  #return numpy array containing the calculated control values (size 3 arrays)

def PID_rotate(yaw_error):
    print("PID Rotate")
    print(" yaw_error = ", yaw_error)
    ble.send_command(CMD.PID_STATUS_INI, "1")
    # ble.send_command(CMD.SET_ORIENT_PID_VALUES, "|3|0.01|0.2")
    ble.send_command(CMD.SET_ORIENT_PID_VALUES, "|4|0.08|1")
    
    ble.send_command(CMD.PID_ROTATE, yaw_error)

def PID_translation(trans):
    '''
    with new battery:
    4.5|0.05|2:
        150 & 0.7
        120 & 0.7

    '''
    print("PID Trans!")
    print(" trans dist = ", trans)
    ble.send_command(CMD.PID_STATUS_INI, "1")
    # ble.send_command(CMD.SET_PID_VALUES, "|10|0.05|4")
    ble.send_command(CMD.SET_FORWARD_SPEED, "150")
    ble.send_command(CMD.SET_FD_CALI, "0.7") # cali_factor * right_power 
    ble.send_command(CMD.PID_TRANS, trans)
    
    
def navigate(waypoints):
    print("Navigating!")
    
    # Reset Plots
    cmdr.reset_plotter()

    # initialization of goal and pose
    unit_convert_factor = 0.3048

    target_index = 0
    target_pt = waypoints[target_index] * unit_convert_factor
    cur_pose = np.array([target_pt[0], target_pt[1], target_pt[2]])    
    
    distance_error_bound = 0.7 * unit_convert_factor
    
    while True:
        cur_dist_error = np.sqrt((cur_pose[0] - target_pt[0])**2 + (cur_pose[1] - target_pt[1])**2)
        print(" cur_dist_err = ", cur_dist_error)
        
        if cur_dist_error < distance_error_bound: # reach the target point -> update the target
            target_index += 1
            if target_index >= len(waypoints): # all the waypoints have been traversed
                break
            target_pt = waypoints[target_index] * unit_convert_factor
            print(" target_pt = ", waypoints[target_index])
            print(" cur_pt = ", np.array(cur_pose)/unit_convert_factor)
            
        # compute control
        [rot_1, trans, rot_2] = compute_control(cur_pose=target_pt, prev_pose=cur_pose)

        print("rot_1 = ", rot_1)
        print("trans = ", trans, "ft / ", trans*304.8, "mm")
        print("rot_2 = ", rot_2)
        
        # implement control
        
        PID_rotate(rot_1)
        time.sleep(5)
        PID_translation(trans)  # unit: ft
        time.sleep(5)
        
        # update the pose
        loc.init_grid_beliefs()
        cur_pose = get_cur_pose()
        print(" cur pose = ", cur_pose)





In [157]:
waypoints = np.array([[-4, -3], [-2, -1], [1, -1], [2, -3], [5, -3], [5, -2], [5, 3], [0, 3], [0, 0]])
# print(np.shape(waypoints))
# print(waypoints[0])
theta_col = np.zeros((9, 1), dtype=int)
# print(theta_col)
aug_waypoints = np.hstack((waypoints, theta_col))   # the waypoints does not have the third theta value col -> initialize as 0
print(aug_waypoints)

[[-4 -3  0]
 [-2 -1  0]
 [ 1 -1  0]
 [ 2 -3  0]
 [ 5 -3  0]
 [ 5 -2  0]
 [ 5  3  0]
 [ 0  3  0]
 [ 0  0  0]]


# Navigation

In [158]:
navigate(waypoints=aug_waypoints)

Navigating!
 cur_dist_err =  0.0
 target_pt =  [-2 -1  0]
 cur_pt =  [-4. -3.  0.]
Compute Ctrl ing
Finish Compute Control
rot_1 =  45.00000000000001
trans =  0.8621045876226389 ft /  262.7694783073803 mm
rot_2 =  -45.00000000000001
PID Rotate
 yaw_error =  45.00000000000001
PID Trans!
 trans dist =  0.8621045876226389
2024-05-14 16:36:29,975 | INFO     |: Initializing beliefs with a Uniform Distribution
2024-05-14 16:36:29,976 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
Get cur pose ing
Start Observation Loop
Start Sending Data!
['Yaw:20.260', 'Tof1:874.0', 'Tof2:788.0']
['Yaw:20.260', 'Tof1:874.0', 'Tof2:788.0']
['Yaw:20.260', 'Tof1:874.0', 'Tof2:788.0']
['Yaw:20.260', 'Tof1:874.0', 'Tof2:788.0']
['Yaw:20.260', 'Tof1:874.0', 'Tof2:790.0']
['Yaw:20.260', 'Tof1:874.0', 'Tof2:790.0']
['Yaw:20.260', 'Tof1:874.0', 'Tof2:790.0']
['Yaw:20.260', 'Tof1:874.0', 'Tof2:790.0']
['Yaw:20.260', 'Tof1:874.0', 'Tof2:790.0']
['Yaw:20.260', 'Tof1:874.0', 'Tof2:790.0']
['Yaw:4

BleakError: disconnected

qt.pointer.dispatch: skipping QEventPoint(id=4 ts=0 pos=0,0 scn=729.547,549.7 gbl=729.547,549.7 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-729.547,-549.7 last=-729.547,-549.7 Δ 729.547,549.7) : no target window
qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=761.38,644.197 gbl=761.38,644.197 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-761.38,-644.197 last=-761.38,-644.197 Δ 761.38,644.197) : no target window
qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=716.146,420.464 gbl=716.146,420.464 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-716.146,-420.464 last=-716.146,-420.464 Δ 716.146,420.464) : no target window
Traceback (most recent call last):
  File "/Users/harrypeng/Desktop/ECE4160/FastRobots-sim-release-release-mirror/src/plotter.py", line 263, in keyPressEvent
    if event.key() == Qt.Key_Escape:
AttributeError: type object 'Qt' has no attribute 'Key_Escape'
Traceback (most recent call last):
  File "/Users/harrypeng/Desktop/ECE4160/FastRobots-sim-releas

# Path Planning

In [ ]:
# from queue import PriorityQueue

# class Graph:
#     def minDistance(self, dist, queue):
#         minimum = float("Inf")
#         min_index = -1
        
#         for i in range(len(dist)):
#             if dist[i] < minimum and (i in queue):
#                 minimum = dist[i]
#                 min_index = i
#         return min_index
    
#     def findpath(self, j):
#         if self.

# PID Hardcode

Optimize the PID control for the position and orientation. Hardcode between waypoints

In [101]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
import pathlib
import os
from utils import load_config_params
from localization_extras import Localization
import time

# The imports below will only work if you copied the required ble-related python files 
# into the notebooks directory
from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import numpy as np

# Setup Logger
LOG = get_logger('demo_notebook.log')
LOG.propagate = False

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2024-05-14 16:22:25,314 | INFO     |: Logger demo_notebook.log already initialized.
2024-05-14 16:22:25,315 | INFO     |: GUI is already running. Shutdown notebook to force restart the GUI.


In [102]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2024-05-14 16:22:25,422 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:42:30:78:92:49


2024-05-14 16:22:29,955 | INFO     |: Connected to c0:42:30:78:92:49


In [131]:
def PID_rotate(yaw_error):
    print("PID Rotate")
    print(" yaw_error = ", yaw_error)
    ble.send_command(CMD.PID_STATUS_INI, "1")
    # ble.send_command(CMD.SET_ORIENT_PID_VALUES, "|3|0.01|0.2")
    ble.send_command(CMD.SET_ORIENT_PID_VALUES, "|4|0.08|1")
    
    ble.send_command(CMD.PID_ROTATE, yaw_error)
    
def PID_translation(trans): # trans unit: ft
    '''
    with new battery:
    4.5|0.05|2:
        150 & 0.7
        120 & 0.7
    4|0.5|1:
        150 & 0.65
        130 & 0.65
    
    '''
    print("PID Trans!")
    print(" trans dist = ", trans, " ft")
    ble.send_command(CMD.PID_STATUS_INI, "1")
    # ble.send_command(CMD.SET_PID_VALUES, "|10|0.05|4")
    
    ble.send_command(CMD.SET_ORIENT_PID_VALUES, "|4|0.08|1")
    ble.send_command(CMD.SET_PID_VALUES, "|4|0.5|3")
    
    ble.send_command(CMD.SET_FORWARD_SPEED, "110")
    ble.send_command(CMD.SET_FD_CALI, "0.6") # cali_factor * right_power 
    ble.send_command(CMD.PID_TRANS, trans)

In [132]:
# ##### Test the PID control precision
# orientation_test = np.array([-45, -45, -90, 45, 45, 270, 180])
# for i in range(len(orientation_test)):
#     PID_rotate(orientation_test[i])
#     time.sleep(3)

In [133]:
# PID_translation(4)  # unit: ft

In [134]:
waypoints = np.array([
    [-4, -3], 
                      [-2, -1], 
                      [1, -1], 
                      [2, -3], 
                      [5, -3], 
                      [5, -2], 
                      [5, 3], 
                      [0, 3], 
                      [0, 0]]) 


In [135]:
pt_orientation = np.array([
    0, 
    45,
    0, np.arctan(-2)*180/np.pi, 0, 90, 90, 180, 270])

In [136]:
print(np.shape(pt_orientation))
print(np.shape(waypoints))

(9,)
(9, 2)


In [137]:
waypoints_with_orientation = np.hstack((waypoints, pt_orientation.reshape(-1, 1)))
print(waypoints_with_orientation)

[[ -4.          -3.           0.        ]
 [ -2.          -1.          45.        ]
 [  1.          -1.           0.        ]
 [  2.          -3.         -63.43494882]
 [  5.          -3.           0.        ]
 [  5.          -2.          90.        ]
 [  5.           3.          90.        ]
 [  0.           3.         180.        ]
 [  0.           0.         270.        ]]


In [138]:
''' data points locations
  angle                   ft
1 45.0 			        2.8284271247461903
2 -45.0 			      3.0
3 -63.43494882292202 	2.23606797749979
4 63.43494882292202 	3.0
5 90.0 			        1.0
6 0.0 			        5.0
7 90.0 			        5.0
8 90.0 			        3.0
'''

' data points locations\n  angle                   ft\n1 45.0 \t\t\t        2.8284271247461903\n2 -45.0 \t\t\t      3.0\n3 -63.43494882292202 \t2.23606797749979\n4 63.43494882292202 \t3.0\n5 90.0 \t\t\t        1.0\n6 0.0 \t\t\t        5.0\n7 90.0 \t\t\t        5.0\n8 90.0 \t\t\t        3.0\n'

In [139]:
## main loop - only need to move 8 groups of (rotation&translation)
# for i in range(np.shape(waypoints_with_orientation)[0] - 1):
for i in range(100):
    print("index = ", i+1)
    # # state_difference  unit: (ft, ft, degree)
    # state_difference = waypoints_with_orientation[i+1] - waypoints_with_orientation[i]
    
    # first change the orientation
    # orientation_change_angle = state_difference[2]
    # PID_rotate(orientation_change_angle)
    orientation_change_angle = input("angle change: ")
    PID_rotate(float(orientation_change_angle))
    time.sleep(2)

    # Then, move 
    # translation_distance = np.linalg.norm(np.array([state_difference[0], state_difference[1]]))
    # PID_translation(translation_distance)  # unit: ft
    
    translation_distance = input("distance change: ")
    PID_translation(float(translation_distance))  # unit: ft
    time.sleep(2)
    
    print("angle change = ", orientation_change_angle, "\t\t\tdistance change = ", translation_distance)
    

index =  1
PID Rotate
 yaw_error =  0.0
PID Trans!
 trans dist =  8.0  ft
angle change =  0 			distance change =  8
index =  2
PID Rotate
 yaw_error =  0.0
PID Trans!
 trans dist =  8.0  ft
angle change =  0 			distance change =  8
index =  3
PID Rotate
 yaw_error =  0.0
PID Trans!
 trans dist =  8.0  ft
angle change =  0 			distance change =  8
index =  4
PID Rotate
 yaw_error =  0.0
PID Trans!
 trans dist =  8.0  ft
angle change =  0 			distance change =  8
index =  5


ValueError: could not convert string to float: ''

2024-05-14 16:33:47,225 | INFO     |: Disconnected from 1EF8FB8D-1A06-FB11-4002-B52B84BE3598


47
